# TD 3 | Computational Statistics | MVA 23-24
### Meilame TAYEBJEE

In [11]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
from numba import jit

## Exercise 1: Hasting-Metropolis within Gibbs – Stochastic Approximation EM

### Data generation

In [55]:
#-----INPUTS-----#
T = 5 # Time horizon

p0 = 10

sigma_t0 = 0.1
sigma_v0 = 0.1

s_t0 = 0.1
s_v0 = 0.1

t0_bar_bar = 1
v0_bar_bar = 1

m = 5
m_ksi = 5
m_tau = 5

v = 1
v_ksi = 1
v_tau = 1

N = 100
k = 21

ts = np.linspace(0, T, k).reshape((1,k))
ts = np.repeat(ts, N, axis = 0)
ts.shape


(100, 21)

In [7]:
#Data generation
def generateData(p0 = p0, ts = ts, sigma_t0 = sigma_t0, sigma_v0 = sigma_v0, s_t0 = s_t0, s_v0 = s_v0, t0_bar_bar = t0_bar_bar, v0_bar_bar = v0_bar_bar, m = m, m_ksi = m_ksi, m_tau = m_tau, v = v, v_ksi= v_ksi, v_tau = v_tau, N = N, k = k):

    t0_bar = np.random.normal(t0_bar_bar, sigma_t0, 1)
    v0_bar = np.random.normal(v0_bar_bar, sigma_v0, 1)

    sigma_ksi_sq = stats.invwishart.rvs(m_ksi, v_ksi**2)
    sigma_tau_sq = stats.invwishart.rvs(m_tau, v_tau**2)
    sigma_sq = stats.invwishart.rvs(m, v**2)

    t0 = np.random.normal(t0_bar, sigma_t0, 1)
    v0 = np.random.normal(v0_bar, sigma_v0, 1)

    epsilon = np.random.normal(0, np.sqrt(sigma_sq), (N, k))
    ksi = np.random.normal(0, np.sqrt(sigma_ksi_sq), (N,1))
    alpha = np.exp(ksi)

    tau = np.random.normal(0, np.sqrt(sigma_tau_sq), (N,1))

    y = p0 + v0*np.repeat(alpha, k, axis =1)*(ts - t0 - np.repeat(tau, k, axis =1)) + epsilon

    return y, alpha, tau, t0, v0, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq, epsilon

### Hastings-Metropolis Sampler

In [80]:
def logTargetPosteriorZ(y, alpha, tau, t0, v0, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq, ts = ts):
    res = np.sum((y - p0 - v0*np.repeat(alpha.reshape((N,1)), k, axis =1)*(ts - t0 - np.repeat(tau.reshape((N,1)), k, axis =1)))**2) / (2*sigma_sq)
    res += np.sum(np.log(alpha))
    res += np.sum(np.square(np.log(alpha)))/(2*sigma_ksi_sq)
    res += np.sum(np.square(tau)) / (2*sigma_tau_sq)
    res += np.square(t0 - t0_bar) / (2*sigma_t0)
    res += np.square(v0 - v0_bar) / (2*sigma_v0)

    return -res



def metropolisHastingSampler(y, alpha_init, tau_init, t0_init, v0_init, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq, sigma_sq_prop, maxIter = 1000):

    alpha = alpha_init
    tau = tau_init
    t0 = t0_init
    v0 = v0_init

    count = 0

    for k in range(maxIter):
        #Proposal
        alpha_prop = np.random.multivariate_normal(alpha.reshape((N,)), sigma_sq_prop * np.identity(N))
        tau_prop = np.random.multivariate_normal(tau.reshape((N,)), sigma_sq_prop * np.identity(N))
        t0_prop = np.random.normal(t0, sigma_sq_prop)
        v0_prop = np.random.normal(v0, sigma_sq_prop)

        #Acceptance probability
        logTargetProposition = logTargetPosteriorZ(y, alpha_prop, tau_prop, t0_prop, v0_prop, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq)
        logTargetCurrent = logTargetPosteriorZ(y, alpha, tau, t0, v0, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq)
        probAcceptance = min(1, np.exp(logTargetProposition - logTargetCurrent))
        
        #Toss the coin !
        u = np.random.uniform(0,1)
        if u < probAcceptance:
            count += 1
            alpha = alpha_prop
            tau = tau_prop
            t0 = t0_prop
            v0 = v0_prop
    
    return alpha, tau, t0, v0, count/maxIter

In [82]:
y, alpha, tau, t0, v0, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq, epsilon = generateData()
metropolisHastingSampler(y, alpha, tau, t0, v0, t0_bar, v0_bar, sigma_ksi_sq, sigma_tau_sq, sigma_sq, 0.5*1e-4)[-1]

0.427